# Exporta qualquer JSON e transforma eles em colunas

#### no trabalho, me deparei com algumas bases que invés de virem colunas normais, elas veem em formato de JSON com inumeras informações, logo, precisei criar um código pronto para extrair de forma automatica as colunas

In [3]:
import pandas as pd
import json

# JSON 1: Dados de Vendas
json_vendas = '''
{
  "vendas": [
    {
      "id_pedido": "PED001",
      "data_pedido": "2024-06-15",
      "cliente": {
        "id": 1,
        "nome": "João",
        "idade": 30,
        "endereco": {
          "tipo": "residencial",
          "logradouro": "Rua das Flores",
          "numero": 123,
          "cidade": "São Paulo",
          "estado": "SP"
        }
      },
      "produtos": [
        {
          "nome": "Camiseta",
          "quantidade": 2,
          "preco_unitario": 25.0
        },
        {
          "nome": "Calça",
          "quantidade": 1,
          "preco_unitario": 100.0
        }
      ]
    }
  ]
}
'''

# JSON 2: Dados de Clientes
json_clientes = '''
{
  "clientes": [
    {
      "id": 2,
      "nome": "Maria",
      "idade": 28,
      "enderecos": [
        {
          "tipo": "residencial",
          "logradouro": "Rua das Acácias",
          "numero": 321,
          "cidade": "Rio de Janeiro",
          "estado": "RJ"
        }
      ]
    }
  ]
}
'''

# Carrega JSONs como dicionários
data_vendas = json.loads(json_vendas)
data_clientes = json.loads(json_clientes)

# Cria DataFrame do Pandas com os JSONs dentro de uma coluna
df = pd.DataFrame({
    'json_data': [data_vendas, data_clientes]
})
display(df)


# Função para "achatamento" completo do JSON
def flatten_json(data):
    # Verifica se o tipo de dado é um dicionário
    if isinstance(data, dict):
        # Inicializa um novo dicionário vazio
        flattened = {}
        # Itera sobre cada chave e valor do dicionário
        for key, value in data.items():
            # Verifica se o valor é um dicionário
            if isinstance(value, dict):
                # Aplica recursivamente a função flatten_json para o valor
                flattened.update(flatten_json(value))
            # Verifica se o valor é uma lista
            elif isinstance(value, list):
                # Itera sobre cada item da lista
                for i, item in enumerate(value):
                    # Aplica recursivamente a função flatten_json para o item
                    flattened.update(flatten_json({f"{key}_{i}": item}))
            else:
                # Se não for um dicionário nem uma lista, adiciona ao dicionário flattened
                flattened[key] = value
        return flattened
    else:
        return {data}


# Aplica a função flatten_json() a cada linha do DataFrame
df['json_data'] = df['json_data'].apply(flatten_json)

# Explode as colunas "json_data" para transformar chaves em colunas
df = pd.json_normalize(df['json_data'])

# Mostra o DataFrame resultante
display(df)


,json_data
0,"{'vendas': [{'id_pedido': 'PED001', 'data_pedi..."
1,"{'clientes': [{'id': 2, 'nome': 'Maria', 'idad..."


,id_pedido,data_pedido,id,nome,idade,tipo,logradouro,numero,cidade,estado,quantidade,preco_unitario
0,PED001,2024-06-15,1,Calça,30,residencial,Rua das Flores,123,São Paulo,SP,1.0,100.0
1,NaN,NaN,2,Maria,28,residencial,Rua das Acácias,321,Rio de Janeiro,RJ,NaN,NaN
